<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_MedmcQADataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Medmcqa.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_MedmcQA"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset Medmcqa")
dataset = load_dataset("openlifescienceai/medmcqa")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

In [43]:
from datasets import load_dataset
dataset_test = load_dataset("json", data_files="/content/test_dataset.json", split="train")

In [46]:
dataset_test[10] ## empty exp for testing dataset

{'id': '473071cf-e645-4318-bb6c-57b6115c2eb5',
 'question': 'Tannerella forsythia is a ?',
 'opa': 'Nonmotile, gram-negative obligate anaerobe',
 'opb': 'Motile, gram-negative obligate anaerobe',
 'opc': 'Nonmotile, gram-negative obligate aerobe',
 'opd': 'Motile, gram-negative obligate aerobe',
 'cop': -1,
 'choice_type': 'single',
 'exp': '',
 'subject_name': 'Dental',
 'topic_name': None}

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

In [ ]:
dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [47]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['question'][0:nrec]
dataset_final_Answer=dataset['exp'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [48]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [49]:
dataset[10]

{'id': 'b3d1eb6c-3909-4011-ad10-d55538b81456',
 'question': 'Naglers reaction is shown by',
 'opa': 'Clostridium tetani',
 'opb': 'Clostridium botulinum',
 'opc': 'Clostridium perfringens',
 'opd': 'Clostridium septicum',
 'cop': 2,
 'choice_type': 'single',
 'exp': "Nagler's reaction - when Clostridium perfringens is grown in media containing 6 % agar, 5% Fildes peptic digest of sheep blood and 20% human serum, with antitoxin spread on one half of plate, colonies in the other half without antitoxin will be surrounded by a zone of opacity. No opacity around the colonies on the half of plate with antitoxin, this is due to the specific neutralization of the alpha-toxin. This specific lecithinase effect, known as the Nagler's reaction. Reference: Textbook of Microbiology; Anathanarayan and paniker's; 10th edition; Page no: 263",
 'subject_name': 'Microbiology',
 'topic_name': 'Bacteriology'}

In [50]:
datasetF

,id,Question,Answer
0,e9ad821a-c438-4965-9f77-760819dfa155,Chronic urethral obstruction due to benign pri...,Chronic urethral obstruction because of urinar...
1,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,Which vitamin is supplied from only animal sou...,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...
2,5c38bea6-787a-44a9-b2df-88f4218ab914,All of the following are surgical options for ...,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba..."
3,cdeedb04-fbe9-432c-937c-d53ac24475de,Following endaerectomy on the right common car...,The central aery of the retina is a branch of ...
4,dc6794a3-b108-47c5-8b1b-3b4931577249,Growth hormone has its effect on growth through?,"Ans. is 'b' i.e., IGI-1GH has two major functi..."
5,5ab84ea8-12d1-47d4-ab22-668ebf01e64c,Scrub typhus is transmitted by: September 2004,Ans. C i.e. Mite
6,a83de6e4-9427-4480-b404-d96621ebb640,Abnormal vascular patterns seen with colposcop...,"Abnormal vascular pattern include punctation, ..."
7,f3bf8583-231b-4b7a-828c-179b0f9ccdd9,Per rectum examination is not a useful test fo...,PILONIDAL SINUS/DISEASE (Jeep Bottom; Driver's...
8,73515f05-e947-4801-8077-3abdeca95c84,Characteristics of Remifentanyl – a) Metabolis...,Remifentanil is the shortest acting opioid due...
9,53f79833-21b0-4336-8ef4-404c687ec807,Hypomimia is ?,Ans. C. Deficit of expression by gestureHypomi...


In [51]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

In [52]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [53]:
nrec=10
eval_dataset[nrec]

{'id': 'b3d1eb6c-3909-4011-ad10-d55538b81456',
 'question': 'Naglers reaction is shown by',
 'opa': 'Clostridium tetani',
 'opb': 'Clostridium botulinum',
 'opc': 'Clostridium perfringens',
 'opd': 'Clostridium septicum',
 'cop': 2,
 'choice_type': 'single',
 'exp': "Nagler's reaction - when Clostridium perfringens is grown in media containing 6 % agar, 5% Fildes peptic digest of sheep blood and 20% human serum, with antitoxin spread on one half of plate, colonies in the other half without antitoxin will be surrounded by a zone of opacity. No opacity around the colonies on the half of plate with antitoxin, this is due to the specific neutralization of the alpha-toxin. This specific lecithinase effect, known as the Nagler's reaction. Reference: Textbook of Microbiology; Anathanarayan and paniker's; 10th edition; Page no: 263",
 'subject_name': 'Microbiology',
 'topic_name': 'Bacteriology'}

In [54]:
eval_dataset[nrec]['question']

'Naglers reaction is shown by'

In [55]:
eval_dataset[nrec]['exp']

"Nagler's reaction - when Clostridium perfringens is grown in media containing 6 % agar, 5% Fildes peptic digest of sheep blood and 20% human serum, with antitoxin spread on one half of plate, colonies in the other half without antitoxin will be surrounded by a zone of opacity. No opacity around the colonies on the half of plate with antitoxin, this is due to the specific neutralization of the alpha-toxin. This specific lecithinase effect, known as the Nagler's reaction. Reference: Textbook of Microbiology; Anathanarayan and paniker's; 10th edition; Page no: 263"

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [93]:
from random import randint
rec = randint(0, len(eval_dataset))

rec = 6

In [ ]:
# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])

#Use the chat template to format the input for the language model
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = eval_dataset[rec]['question']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [95]:
outputs

[{'generated_text': 'Abnormal vascular patterns seen with colposcopy in case of cervical intraepithelial neoplasia  are all except [/INST] Abnormal vascular pattern include punctation, mosaicism and atypical vessels. These are usually seen in CIN 2 and 3. Ref: Shaw Gynecology 17 e pg 340. '}]

In [96]:
oanswer=eval_dataset[rand_idx]['exp']
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
#rand_idx = randint(0, len(eval_dataset))

print(f"Question:\n{eval_dataset[rand_idx]['question']}")
print()
print(f"Original Answer:\n{oanswer}")
print()
print(f"Generated Answer:\n{ganswer}")

from difflib import SequenceMatcher

  # Utility function to compute similarity
def similar(str1, str2):
  return SequenceMatcher(None, str1, str2).ratio()

# using SequenceMatcher.ratio()

# similarity between strings
similarity = similar(oanswer, ganswer)

# printing the result
print()
print ("The similarity between 2 strings is : " + str(similarity))

Question:
Abnormal vascular patterns seen with colposcopy in case of cervical intraepithelial neoplasia  are all except

Original Answer:
Abnormal vascular pattern include punctation, mosaicism and atypical vessels.

Generated Answer:
Abnormal vascular pattern include punctation, mosaicism and atypical vessels. These are usually seen in CIN 2 and 3. Ref: Shaw Gynecology 17 e pg 340.

The similarity between 2 strings is : 0.6784140969162996


In [ ]:
prompt='capital of russia?'

outputs = generation_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7,
                              top_k=50, top_p=0.9, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [ ]:
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

Generated Answer:
Ans. is 'd' i.e., Moscow


In [ ]:
q=eval_dataset[1]['question']

In [97]:
number_of_eval_samples = 10
eval_dataset.shuffle().select(range(number_of_eval_samples))

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 10
})

In [98]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [170]:
from tqdm import tqdm
import re

# Load our test dataset
#eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")
eval_dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")
#eval_dataset = load_dataset("json", data_files="/content/validation_dataset.json", split="train")

def evaluate(sample):

    prompt = sample['question']
    answer=sample['exp']

    if answer == None:
       answer_original = 'Not possible to get or use'

    answer_original = re.sub("\n", "", str(answer))


    if len(answer_original)==0:
        answer_original='Not possible to get or use'

    str_question= sample['question']

    prompt = re.sub("\n", "", str_question)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+9:].strip()
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

    oanswer=sample['exp']

    if oanswer == None:
       oanswer = 'Not possible to get or use'

    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
    #print()
    #print(f"Question:\n{sample['question']}")
    #print(f"Original Answer:\n{sample['exp']}")
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+9:].strip()}")
    #print()

    # Python3 code to demonstrate
    # similarity between strings
    # using SequenceMatcher.ratio()
    from difflib import SequenceMatcher

    # Utility function to compute similarity
    def similar(str1, str2):
      return SequenceMatcher(None, str1, str2).ratio()

    # using SequenceMatcher.ratio()
    # similarity between strings
    res = similar(oanswer, ganswer)

    # printing the result
    #print ("The similarity between 2 strings is : " + str(res))



    if  res>0.30:
          print ()
          print ("The similarity between 2 strings is : " + str(res))
          print(f"Question:\n{sample['question']}")
          print(f"Original Answer:\n{answer_original}")
          print(f"Generated Answer:\n{ganswer}")
          print ()
          return 1
    else:
          #print(f"Original Answer:\n{answer_original}")
          return 0

success_rate = []
number_of_eval_samples = 10
# iterate over eval dataset and predict

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
for n in range(number_of_eval_samples):
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")


The similarity between 2 strings is : 0.338368580060423
Question:
Which vitamin is supplied from only animal source:
Original Answer:
Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off.
Generated Answer:
Ans. (c) Vitamin B12(Ref: Harrison 19th/pg 640, 18th/pg 659)Vitamin B12 (Cobalamin) is synthesized solely by microorganisms. It is found in foods of animal origin, such as meat, fish, and dairy products.


The similarity between 2 strings is : 0.4155161078238001
Question:
All of the following are surgical options for morbid obesity except -
Original Answer:
Ans. is 'd' i.e., Roux en Y

In [171]:
# Load our test dataset
#eval_dataset = load_dataset("json", data_files="/content/validation_dataset.json", split="train")
eval_dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

In [ ]:
from tqdm import tqdm
import re
from difflib import SequenceMatcher

def generate_answer(prompt):
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    return outputs
# Utility function to compute similarity
def similar(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

def evaluate(sample):

    prompt = sample['question']
    answer=sample['exp']

    if answer == None:
       answer_original = 'Not possible to get or use'

    answer_original = re.sub("\n", "", str(answer))


    if len(answer_original)==0:
        answer_original='Not possible to get or use'

    str_question= sample['question']

    prompt = re.sub("\n", "", str_question)


    oanswer=sample['exp']

    outputs = generate_answer(prompt)
    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()

    if oanswer == None:
       oanswer = 'Not possible to get or use'

    # similarity between strings
    res = similar(oanswer,ganswer)
    #del outputs, res,
    # printing the result
    #print ("The similarity between 2 strings is : " + str(res))


    if  res>=0.30:
          #print ()
          #print ("The similarity between 2 strings is : " + str(res))
          #print(f"Question:\n{sample['question']}")
          #print(f"Original Answer:\n{answer_original}")
          #print(f"Generated Answer:\n{ganswer}")
          #print ()
          return 1
    else:
          #print(f"Original Answer:\n{answer_original}")
          return 0
    del outputs, res,

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
for n in range(number_of_eval_samples):
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

In [112]:
#print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

#Accuracy (Eval dataset and predict) for a sample of 1000: 3.50% EPOCH=1

#Accuracy (Eval dataset and predict) for a sample of 100: 2.00% EPOCH=10 /content/test_dataset.json

# /content/train_dataset.json" Accuracy (Eval dataset and predict) for a sample of 100: 51.00%

# /content/train_dataset.json" Accuracy (Eval dataset and predict) for a sample of 1000: 48.00%